<a href="https://colab.research.google.com/github/olegkleiman/udemy_nlp/blob/main/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#from sentence_transformers import SentenceTransformer

In [139]:
import nltk
stopwords = nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('hebrew'))
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'אבל',
 'או',
 'אולי',
 'אותה',
 'אותו',
 'אותי',
 'אותך',
 'אותם',
 'אותן',
 'אותנו',
 'אז',
 'אחר',
 'אחרות',
 'אחרי',
 'אחרים',
 'אחרת',
 'אי',
 'איזה',
 'איך',
 'אין',
 'איפה',
 'איתה',
 'איתו',
 'איתי',
 'איתך',
 'איתכם',
 'איתכן',
 'איתם',
 'איתן',
 'איתנו',
 'אך',
 'אל',
 'אלה',
 'אלו',
 'אם',
 'אנחנו',
 'אני',
 'אס',
 'אף',
 'אצל',
 'אשר',
 'את',
 'אתה',
 'אתכם',
 'אתכן',
 'אתם',
 'אתן',
 'באיזו מידה',
 'באמצע',
 'באמצעות',
 'בגלל',
 'בין',
 'בלי',
 'במידה',
 'במקום שבו',
 'ברם',
 'בשביל',
 'בשעה ש',
 'בתוך',
 'גם',
 'דרך',
 'הוא',
 'היא',
 'היה',
 'היכן',
 'היתה',
 'היתי',
 'הם',
 'הן',
 'הנה',
 'הסיבה שבגללה',
 'הרי',
 'ואילו',
 'ואת',
 'זאת',
 'זה',
 'זות',
 'יהיה',
 'יוכל',
 'יוכלו',
 'יותר',
 'יכול',
 'יכולה',
 'יכולות',
 'יכולים',
 'יכל',
 'יכלה',
 'יכלו',
 'יש',
 'כאן',
 'כאשר',
 'כולם',
 'כולן',
 'כזה',
 'כי',
 'כיצד',
 'כך',
 'ככה',
 'כל',
 'כלל',
 'כמו',
 'כן',
 'כפי',
 'כש',
 'לא',
 'לאו',
 'לאיזו תכלית',
 'לאן',
 'לבין',
 'לה',
 'להיות',
 'להם',
 'להן',
 'לו',
 'לי

In [132]:
corpus = [[
     "יום הבחירות הוא יום שבתון. ביום הבחירות הקלפיות יפעלו בין השעות 07:00 - 22:00.",
     "ימים אלו החל לפעול בעיר מרכז מבקרים חוויתי שיציג את תל אביב-יפו המתחדשת. \
 הפעילות במרכז כוללת חידון תחרותי, חווית תוכן מרהיבה המתארת את השינוי המרגש שהעיר עוברת בתחבורה ובהתחדשות עירונית, ולסיום עמדות צילום למזכרת. \
 המרכז יעבור משכונה לשכונה ובכך יגיע לכל תושבות ותושבי העיר כמעט עד הבית. \
 מיקומים תאריך קרובים: "
 "שלוש שנים לאחר שקיבלה תל אביב מעמד של מועצה עירונית, ב-24 בינואר 1924, נערכו הבחירות הראשונות למועצה.",
             "ב-1955 החליטה מפלגת מפאי להעמיד את גולדה מאיר בראש סיעתה בחירות לעיריית תל-אביב יפו.",
             "בין תאריך 19-23.7 נערכו שישה מפגשי חשיבה ודיון בהשתתפות כ-80 תושבים בסוגיות רלוונטיות למסמך המדיניות, במסגרת סקירת מצב קיים בשכונה. לאור מגבלות משרד הבריאות, המפגשים התקיימו בצורה מקוונת דרך פלטפורמת Zoom. ",
"הנחות  ופטורים בתשלום ארנונה נחלקים לשלושה סוגים: \
הנחות אישיות: ניתנות לתושבי העיר ולבעלי העסקים בעיר בהתאם לקריטריונים הקשורים למחזיק בנכס שנקבעו בחוקים השונים. \
הנחות לנכס: ניתנות לתושבי העיר ולבעלי העסקים בעיר בהתאם לקריטריונים הקשורים לנכס. \
פטורים נוספים מתשלום ארנונה, מלאים או חלקיים למוסדות שונים ניתנים לפי פקודת מסי העירייה ומסי הממשלה (פיטורין) - 1938 ",
"מועצת העירייה מטילה את הארנונה הכללית לשנת הכספים במסגרת צו הארנונה, מתוקף הסמכות המוקנית לה בחוק הסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב),"
"מתייחסים אך ורק לשינויים בצו הארנונה המובאים להחלטת מועצת העירייה. דברי ההסבר אינם מתייחסים לשינויים המחוייבים על פי דין. במסגרת החלטתה על הטלת ארנונה קובעת המועצה אם לאשר שינויים אלה במלואם או בחלקם או לדחותם. חלק מהשינויים מותנים, כאמור, על פי דין בקבלת אישור שר הפנים ושר האוצר. הנוסח הקובע של צו הארנונה הוא הנוסח כפי שקבעה מועצת העירייה, בכפוף לאישור חריג של שר הפנים ושר האוצר, ככל שנדרש."
  ],
   ["https://www.tel-aviv.gov.il/Pages/MainItemPage.aspx?WebID=3af57d92-807c-43c5-8d5f-6fd455eb2776&ListID=81e17809-311d-4bba-9bf1-2363bb9debcd&ItemID=2173",
      "2",
      "3",
      "4",
      "5",
      "6",
  ]
]
print(corpus[0])
# Second row - only links
#len(corpus[1])
corpus




['יום הבחירות הוא יום שבתון. ביום הבחירות הקלפיות יפעלו בין השעות 07:00 - 22:00.', 'ימים אלו החל לפעול בעיר מרכז מבקרים חוויתי שיציג את תל אביב-יפו המתחדשת.  הפעילות במרכז כוללת חידון תחרותי, חווית תוכן מרהיבה המתארת את השינוי המרגש שהעיר עוברת בתחבורה ובהתחדשות עירונית, ולסיום עמדות צילום למזכרת.  המרכז יעבור משכונה לשכונה ובכך יגיע לכל תושבות ותושבי העיר כמעט עד הבית.  מיקומים תאריך קרובים: שלוש שנים לאחר שקיבלה תל אביב מעמד של מועצה עירונית, ב-24 בינואר 1924, נערכו הבחירות הראשונות למועצה.', 'ב-1955 החליטה מפלגת מפאי להעמיד את גולדה מאיר בראש סיעתה בחירות לעיריית תל-אביב יפו.', 'בין תאריך 19-23.7 נערכו שישה מפגשי חשיבה ודיון בהשתתפות כ-80 תושבים בסוגיות רלוונטיות למסמך המדיניות, במסגרת סקירת מצב קיים בשכונה. לאור מגבלות משרד הבריאות, המפגשים התקיימו בצורה מקוונת דרך פלטפורמת Zoom. ', 'הנחות  ופטורים בתשלום ארנונה נחלקים לשלושה סוגים: הנחות אישיות: ניתנות לתושבי העיר ולבעלי העסקים בעיר בהתאם לקריטריונים הקשורים למחזיק בנכס שנקבעו בחוקים השונים. הנחות לנכס: ניתנות לתושבי העיר ולבעלי ה

AttributeError: ignored

Standardized corpus

In [116]:
#https://digitelmobile.tel-aviv.gov.il/SharepointData/api/ListData/%D7%94%D7%98%D7%91%D7%95%D7%AA/%D7%9B%D7%9C%20%D7%94%D7%A4%D7%A8%D7%99%D7%98%D7%99%D7%9D

df = pd.DataFrame({'texts': corpus[0], 'links': corpus[1]})
df.head()


,texts,links
0,יום הבחירות הוא יום שבתון. ביום הבחירות הקלפיו...,https://www.tel-aviv.gov.il/Pages/MainItemPage...
1,ימים אלו החל לפעול בעיר מרכז מבקרים חוויתי שיצ...,2
2,ב-1955 החליטה מפלגת מפאי להעמיד את גולדה מאיר ...,3
3,בין תאריך 19-23.7 נערכו שישה מפגשי חשיבה ודיון...,4
4,הנחות ופטורים בתשלום ארנונה נחלקים לשלושה סוג...,5


In [117]:
df.memory_usage()

Index    128
texts     48
links     48
dtype: int64

In [95]:
find_nearest_to ="תאריך הבחירות"
input = "הטובות ארנונה"
#find_nearest_to = input.split(sep=" ")

Learn vocabulary and idf, return TF-IDF-weighted document-term matrix.

In [96]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus[0])

Transform document(s) to document-term matrix.

In [97]:
new_entry = vectorizer.transform([find_nearest_to])

Print out the vocabulary

In [98]:
vectorizer.get_feature_names_out()

array(['00', '07', '19', '1924', '1938', '1955', '22', '23', '24', '80',
       'zoom', 'אביב', 'או', 'אינם', 'אישור', 'אישיות', 'אך', 'אלה',
       'אלו', 'אם', 'ארנונה', 'את', 'בהשתתפות', 'בהתאם', 'בחוק', 'בחוקים',
       'בחירות', 'בחלקם', 'ביום', 'בין', 'בינואר', 'בכפוף', 'במלואם',
       'במסגרת', 'במרכז', 'במשק', 'בנכס', 'בסוגיות', 'בעיר', 'בצו',
       'בצורה', 'בקבלת', 'בראש', 'בשכונה', 'בתחבורה', 'בתשלום', 'גולדה',
       'דברי', 'דין', 'דרך', 'האוצר', 'הארנונה', 'הבחירות', 'הבית',
       'הבריאות', 'ההסבר', 'הוא', 'החל', 'החלטתה', 'החליטה', 'הטלת',
       'הכללית', 'הכספים', 'המדינה', 'המדיניות', 'המובאים', 'המועצה',
       'המוקנית', 'המחוייבים', 'הממשלה', 'המפגשים', 'המרגש', 'המרכז',
       'המתארת', 'המתחדשת', 'הנוסח', 'הנחות', 'הסדרים', 'הסמכות', 'העיר',
       'העירייה', 'העסקים', 'הפנים', 'הפעילות', 'הקובע', 'הקלפיות',
       'הקשורים', 'הראשונות', 'השונים', 'השינוי', 'השעות', 'התקיימו',
       'התקציב', 'ובהתחדשות', 'ובכך', 'ודיון', 'ולבעלי', 'ולסיום', 'ומסי',
       '

If x and y are column vectors, their linear kernel is: \K(x,y)=x^{T}y

In [99]:
cosine_similarities = linear_kernel(new_entry, X).flatten()
cosine_similarities


array([0.26580627, 0.13960493, 0.        , 0.10466709, 0.        ,
       0.        ])

In [112]:
df['cos_similarities'] = cosine_similarities
df = df.sort_values(by=['cos_similarities'], ascending=[0])
df


,texts,links,cos_similarities
0,יום הבחירות הוא יום שבתון. ביום הבחירות הקלפיו...,https://www.tel-aviv.gov.il/Pages/MainItemPage...,0.265806
1,ימים אלו החל לפעול בעיר מרכז מבקרים חוויתי שיצ...,2,0.139605
3,בין תאריך 19-23.7 נערכו שישה מפגשי חשיבה ודיון...,4,0.104667
2,ב-1955 החליטה מפלגת מפאי להעמיד את גולדה מאיר ...,3,0.000000
4,הנחות ופטורים בתשלום ארנונה נחלקים לשלושה סוג...,5,0.000000
5,מועצת העירייה מטילה את הארנונה הכללית לשנת הכס...,6,0.000000


In [113]:
df

,texts,links,cos_similarities
0,יום הבחירות הוא יום שבתון. ביום הבחירות הקלפיו...,https://www.tel-aviv.gov.il/Pages/MainItemPage...,0.265806
1,ימים אלו החל לפעול בעיר מרכז מבקרים חוויתי שיצ...,2,0.139605
3,בין תאריך 19-23.7 נערכו שישה מפגשי חשיבה ודיון...,4,0.104667
2,ב-1955 החליטה מפלגת מפאי להעמיד את גולדה מאיר ...,3,0.000000
4,הנחות ופטורים בתשלום ארנונה נחלקים לשלושה סוג...,5,0.000000
5,מועצת העירייה מטילה את הארנונה הכללית לשנת הכס...,6,0.000000


In [115]:
for index, row in df[0:3].iterrows():
  if row['cos_similarities'] > 0:
    print(row['texts'],  row['links'], row['cos_similarities'])

יום הבחירות הוא יום שבתון. ביום הבחירות הקלפיות יפעלו בין השעות 07:00 - 22:00. https://www.tel-aviv.gov.il/Pages/MainItemPage.aspx?WebID=3af57d92-807c-43c5-8d5f-6fd455eb2776&ListID=81e17809-311d-4bba-9bf1-2363bb9debcd&ItemID=2173 0.26580627232293486
ימים אלו החל לפעול בעיר מרכז מבקרים חוויתי שיציג את תל אביב-יפו המתחדשת.  הפעילות במרכז כוללת חידון תחרותי, חווית תוכן מרהיבה המתארת את השינוי המרגש שהעיר עוברת בתחבורה ובהתחדשות עירונית, ולסיום עמדות צילום למזכרת.  המרכז יעבור משכונה לשכונה ובכך יגיע לכל תושבות ותושבי העיר כמעט עד הבית.  מיקומים תאריך קרובים: שלוש שנים לאחר שקיבלה תל אביב מעמד של מועצה עירונית, ב-24 בינואר 1924, נערכו הבחירות הראשונות למועצה. 2 0.13960492755178525
בין תאריך 19-23.7 נערכו שישה מפגשי חשיבה ודיון בהשתתפות כ-80 תושבים בסוגיות רלוונטיות למסמך המדיניות, במסגרת סקירת מצב קיים בשכונה. לאור מגבלות משרד הבריאות, המפגשים התקיימו בצורה מקוונת דרך פלטפורמת Zoom.  4 0.1046670907303087
